**改編自〈sovits3.0一键脚本（小狼躺平了，所以是深夜诗人修改版本,已更新32k/48k分支切换）.ipynb〉**

https://colab.research.google.com/drive/1rCUOOVG7-XQlVZuWRAj5IpGrMM8t07pE?usp=sharing

做了大幅(?)調整使其可用在vast.ai的機台上頭（運作映像為nvidia/cuda:12.0.1-runtime-ubuntu22.04 `00602486357e8c88c07157ddcd8fb2d7`），包含各種版本差異造成的坑。

原Repo `innnky/so-vits-svc` 因為某些因素不見了，所以使用我自己的fork，預訓練模型採用 `TachibanaKimika/so-vits-svc-4.0-models` 提供的模型。

註解繁中化。

In [1]:
# 查看顯示卡
!nvidia-smi

Tue Mar  7 09:20:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:03:00.0 Off |                  N/A |
|  0%   42C    P8     4W / 420W |      1MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from pathlib import Path

root = Path('/root')
sovits = root / 'so-vits-svc'

In [5]:
#@title Clone github Repo
!git clone https://github.com/gimi65536/so-vits-svc -b 4.0
%cd {sovits}

fatal: destination path 'so-vits-svc' already exists and is not an empty directory.
/root/so-vits-svc


In [ ]:
# 安裝Dependencies
!apt-get install build-essential zip -y
!pip install pyworld praat-parselmouth librosa==0.9.2 fairseq


In [7]:
# 下載必要模型文件
!wget -P hubert/ https://huggingface.co/TachibanaKimika/so-vits-svc-4.0-models/resolve/main/others/checkpoint_best_legacy_500.pt

--2023-03-07 09:24:14--  https://huggingface.co/innnky/contentvec/resolve/main/checkpoint_best_legacy_500.pt
Resolving huggingface.co (huggingface.co)... 54.235.118.239, 3.231.67.228, 2600:1f18:147f:e800:671:b733:ecf3:a585, ...
Connecting to huggingface.co (huggingface.co)|54.235.118.239|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/0a/5f/0a5f89ff9058782f4b7a207d38e0914d63f6f55e8fc1fad0c3bf68b546039f2d/60d936ec5a566776fc392e69ad8b630d14eb588111233fe313436e200a7b187b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27checkpoint_best_legacy_500.pt%3B+filename%3D%22checkpoint_best_legacy_500.pt%22%3B&Expires=1678439717&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzBhLzVmLzBhNWY4OWZmOTA1ODc4MmY0YjdhMjA3ZDM4ZTA5MTRkNjNmNmY1NWU4ZmMxZmFkMGMzYmY2OGI1NDYwMzlmMmQvNjBkOTM2ZWM1YTU2Njc3NmZjMzkyZTY5YWQ4YjYzMGQxNGViNTg4MTExMjMzZmUzMTM0MzZlMjAwYTdiMTg3Yj9yZXNwb25zZS1jb250

# 資料集預處理預處理

該處理腳本可以一次性預處理多位角色，並且生成多角色的filelists以及對應的配置文件

只需要將你的資料集按照以下文件結構放到dataset_raw資料夾下


```
dataset_raw
├───speaker0
│   ├───xxx1-xxx1.wav
│   ├───...
│   └───Lxx-0xx8.wav
└───speaker1
    ├───xx2-0xxx2.wav
    ├───...
    └───xxx7-xxx007.wav

```




In [3]:
# 設定訓練資料集備份與還原的路徑，預設為/root
DATA_PATH = root

In [9]:
# 資料集名稱（壓縮檔名稱不帶zip）
DATASETNAME = "mika"  #@param {type:"string"}
# 壓縮檔路徑
ZIP_PATH = root
ZIP_NAME = ZIP_PATH / DATASETNAME

!unzip -d {sovits / 'dataset_raw'} {ZIP_NAME}.zip

Archive:  /root/mika.zip
   creating: /root/so-vits-svc/dataset_raw/mika_voice/
  inflating: /root/so-vits-svc/dataset_raw/mika_voice/157766.wav  
  inflating: /root/so-vits-svc/dataset_raw/mika_voice/182536.wav  
  inflating: /root/so-vits-svc/dataset_raw/mika_voice/183845.wav  
  inflating: /root/so-vits-svc/dataset_raw/mika_voice/192622.wav  
  inflating: /root/so-vits-svc/dataset_raw/mika_voice/195410.wav  
  inflating: /root/so-vits-svc/dataset_raw/mika_voice/195990.wav  
  inflating: /root/so-vits-svc/dataset_raw/mika_voice/199358.wav  
  inflating: /root/so-vits-svc/dataset_raw/mika_voice/20150.wav  
  inflating: /root/so-vits-svc/dataset_raw/mika_voice/206410.wav  
  inflating: /root/so-vits-svc/dataset_raw/mika_voice/215752.wav  
  inflating: /root/so-vits-svc/dataset_raw/mika_voice/215796.wav  
  inflating: /root/so-vits-svc/dataset_raw/mika_voice/225893.wav  
  inflating: /root/so-vits-svc/dataset_raw/mika_voice/229728.wav  
  inflating: /root/so-vits-svc/dataset_raw/mika_vo

In [10]:
# 重取樣到44100Hz
!python3 resample.py

./dataset_raw/mika_voice
69it [00:00, 264.65it/s]


In [11]:
# 劃分訓練集、生成配置文件
!python3 preprocess_flist_config.py

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 159.01it/s]
Writing ./filelists/train.txt
100%|███████████████████████████████████████| 65/65 [00:00<00:00, 711827.05it/s]
Writing ./filelists/val.txt
100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 50840.05it/s]
Writing ./filelists/test.txt
100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 52758.54it/s]
Writing configs/config.json


In [14]:
# 生成hubert和f0
!python3 preprocess_hubert_f0.py

[69]
Loading hubert for content...
load model(s) from hubert/checkpoint_best_legacy_500.pt
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/root/so-vits-svc, universal_newlines=False, shell=None, istream=None)
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/root/so-vits-svc, universal_newlines=False, shell=None, istream=None)
INFO:fairseq.tasks.text_to_speech:Please install tensorboardX: pip install tensorboardX
INFO:fairseq.tasks.hubert_pretraining:current directory is /root/so-vits-svc
INFO:fairseq.tasks.hubert_pretraining:HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
INFO:fairseq.models.hubert.hubert:HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_

In [7]:
# 至此，數據集預處理製作完畢，將資料集和相關文件保存到so-vits-svc資料夾中並自行下載，方便下次訓練使用

# 壓縮dataset資料夾
!zip -r dataset.zip dataset
# 自訂壓縮檔名稱（不含zip）
dataset_name_drive = "44kmika_dataset"
DATASET_PATH_DRIVE = DATA_PATH / dataset_name_drive
!mkdir -p {DATASET_PATH_DRIVE}

!cp dataset.zip "{DATASET_PATH_DRIVE}"
!cp configs/config.json "{DATASET_PATH_DRIVE}"
!cp filelists/train.txt "{DATASET_PATH_DRIVE}"
!cp filelists/test.txt "{DATASET_PATH_DRIVE}"
!cp filelists/val.txt "{DATASET_PATH_DRIVE}"

# 自行下載壓縮檔{DATA_PATH / dataset_name_drive}.zip

  adding: dataset/ (stored 0%)
  adding: dataset/44k/ (stored 0%)
  adding: dataset/44k/mika_voice/ (stored 0%)
  adding: dataset/44k/mika_voice/183845.wav (deflated 16%)
  adding: dataset/44k/mika_voice/195410.wav (deflated 6%)
  adding: dataset/44k/mika_voice/157766.wav (deflated 60%)
  adding: dataset/44k/mika_voice/199358.wav (deflated 4%)
  adding: dataset/44k/mika_voice/182536.wav (deflated 17%)
  adding: dataset/44k/mika_voice/20150.wav (deflated 10%)
  adding: dataset/44k/mika_voice/195990.wav (deflated 3%)
  adding: dataset/44k/mika_voice/215752.wav (deflated 4%)
  adding: dataset/44k/mika_voice/206410.wav (deflated 25%)
  adding: dataset/44k/mika_voice/236088.wav (deflated 5%)
  adding: dataset/44k/mika_voice/225893.wav (deflated 2%)
  adding: dataset/44k/mika_voice/229728.wav (deflated 5%)
  adding: dataset/44k/mika_voice/192622.wav (deflated 26%)
  adding: dataset/44k/mika_voice/244446.wav (deflated 36%)
  adding: dataset/44k/mika_voice/215796.wav (deflated 4%)
  adding: da

In [4]:
# 已經預處理過資料集的話，就可以跳過預處理部分，直接解壓縮處理過的資料以及配置文件

# 壓縮檔名稱（不含zip）
back_up_name = "44kmika_dataset"
BACK_UP_DATASET_PATH = DATA_PATH / back_up_name
!unzip {BACK_UP_DATASET_PATH / 'dataset.zip'} -d .
!cp {BACK_UP_DATASET_PATH / 'config.json'} configs/config.json 
!cp {BACK_UP_DATASET_PATH / 'val.txt'} filelists/val.txt
!cp {BACK_UP_DATASET_PATH / 'train.txt'} filelists/train.txt
!cp {BACK_UP_DATASET_PATH / 'test.txt'} filelists/test.txt

# 將之前訓練過的模型G_xx.pth與D_xx.pth放到 logs/44k 底下……

Archive:  /root/48kmika_dataset/dataset.zip
replace /content/so-vits-svc/dataset/48k/mika_voice/183845.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


# 訓練

In [16]:
# 如果是首次訓練，則需事先下載預訓練模型
!wget -P logs/44k/ https://huggingface.co/TachibanaKimika/so-vits-svc-4.0-models/resolve/main/others/G_0.pth
!wget -P logs/44k/ https://huggingface.co/TachibanaKimika/so-vits-svc-4.0-models/resolve/main/others/D_0.pth

--2023-03-07 09:39:05--  https://huggingface.co/innnky/sovits_pretrained/resolve/main/sovits4/G_0.pth
Resolving huggingface.co (huggingface.co)... 54.235.118.239, 3.231.67.228, 2600:1f18:147f:e800:671:b733:ecf3:a585, ...
Connecting to huggingface.co (huggingface.co)|54.235.118.239|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/b8/68/b8680b5490e63bf5f140618c3ee2e5ccd0731956a87ab87a228c08398cd8e03c/20a327c54e5731bed377bd38404bc32ab98e66a1b2777b0af4cc034d4d6914b0?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27G_0.pth%3B+filename%3D%22G_0.pth%22%3B&Expires=1678441146&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2I4LzY4L2I4NjgwYjU0OTBlNjNiZjVmMTQwNjE4YzNlZTJlNWNjZDA3MzE5NTZhODdhYjg3YTIyOGMwODM5OGNkOGUwM2MvMjBhMzI3YzU0ZTU3MzFiZWQzNzdiZDM4NDA0YmMzMmFiOThlNjZhMWIyNzc3YjBhZjRjYzAzNGQ0ZDY5MTRiMD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRpb24iOnsiRGF0ZUx

In [17]:
!python3 train.py -c configs/config.json -m 44k

INFO:44k:{'train': {'log_interval': 200, 'eval_interval': 800, 'seed': 1234, 'epochs': 10000, 'learning_rate': 0.0001, 'betas': [0.8, 0.99], 'eps': 1e-09, 'batch_size': 6, 'fp16_run': False, 'lr_decay': 0.999875, 'segment_size': 10240, 'init_lr_ratio': 1, 'warmup_epochs': 0, 'c_mel': 45, 'c_kl': 1.0, 'use_sr': True, 'max_speclen': 512, 'port': '8001', 'keep_ckpts': 0}, 'data': {'training_files': 'filelists/train.txt', 'validation_files': 'filelists/val.txt', 'max_wav_value': 32768.0, 'sampling_rate': 44100, 'filter_length': 2048, 'hop_length': 512, 'win_length': 2048, 'n_mel_channels': 80, 'mel_fmin': 0.0, 'mel_fmax': 22050}, 'model': {'inter_channels': 192, 'hidden_channels': 192, 'filter_channels': 768, 'n_heads': 2, 'n_layers': 6, 'kernel_size': 3, 'p_dropout': 0.1, 'resblock': '1', 'resblock_kernel_sizes': [3, 7, 11], 'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]], 'upsample_rates': [8, 8, 2, 2, 2], 'upsample_initial_channel': 512, 'upsample_kernel_sizes': [16, 16, 4,